In [32]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [33]:
coupon_list_train = pd.read_csv('input/coupon_list_train.csv')
coupon_list_test = pd.read_csv('input/coupon_list_test.csv')
user_list = pd.read_csv('input/user_list.csv')
coupon_purchases_train = pd.read_csv("input/coupon_detail_train.csv")

coupon_area_train = pd.read_csv('input/coupon_area_train.csv')
coupon_area_test = pd.read_csv('input/coupon_area_test.csv')
coupon_visit_train = pd.read_csv('input/coupon_visit_train.csv')

In [34]:
coupon_list_train

,CAPSULE_TEXT,GENRE_NAME,PRICE_RATE,CATALOG_PRICE,DISCOUNT_PRICE,DISPFROM,DISPEND,DISPPERIOD,VALIDFROM,VALIDEND,...,USABLE_DATE_THU,USABLE_DATE_FRI,USABLE_DATE_SAT,USABLE_DATE_SUN,USABLE_DATE_HOLIDAY,USABLE_DATE_BEFORE_HOLIDAY,large_area_name,ken_name,small_area_name,COUPON_ID_hash
0,グルメ,グルメ,50,3000,1500,2011-07-08 12:00:00,2011-07-09 12:00:00,1,2011-07-10,2011-12-08,...,1.0,0.0,0.0,1.0,1.0,0.0,関東,埼玉県,埼玉,6b263844241eea98c5a97f1335ea82af
1,グルメ,グルメ,51,2080,1000,2011-07-01 12:00:00,2011-07-02 12:00:00,1,2011-07-03,2011-12-04,...,1.0,1.0,1.0,1.0,1.0,1.0,関東,千葉県,千葉,cc031f250e8bad1e24060263b9fc0ddd
2,グルメ,グルメ,50,7000,3500,2011-07-12 12:00:00,2011-07-15 12:00:00,3,2011-07-16,2012-01-11,...,1.0,1.0,1.0,1.0,1.0,1.0,関東,千葉県,千葉,ba5e9b7453ca52ff711635a5d2e8102d
3,グルメ,グルメ,50,3000,1500,2011-07-09 12:00:00,2011-07-11 12:00:00,2,2011-07-12,2011-12-01,...,1.0,0.0,0.0,1.0,1.0,1.0,関東,千葉県,千葉,3e1ffbedca3569f9e8032d401e8cb4e6
4,グルメ,グルメ,50,2000,1000,2011-07-05 12:00:00,2011-07-06 12:00:00,1,2011-07-07,2011-12-30,...,1.0,0.0,0.0,1.0,1.0,0.0,関東,千葉県,千葉,782934b6c815b4030ea204eef7d4a734
5,グルメ,グルメ,50,4000,2000,2011-07-24 12:00:00,2011-07-26 12:00:00,2,2011-07-27,2011-10-30,...,1.0,1.0,1.0,1.0,1.0,1.0,関東,千葉県,千葉,439db4336f84d4e4a13290fc0d7e6c4e
6,グルメ,グルメ,51,3495,1700,2011-09-03 12:00:00,2011-09-05 12:00:00,2,2011-09-07,2012-02-28,...,1.0,1.0,1.0,1.0,1.0,1.0,関東,東京都,新宿・高田馬場・中野・吉祥寺,c9144a97ed61cb778414b758c704d828
7,グルメ,グルメ,51,3100,1500,2011-08-02 12:00:00,2011-08-04 12:00:00,2,2011-08-05,2012-01-31,...,1.0,1.0,1.0,0.0,1.0,1.0,関西,京都府,京都,2b00d8dfc1a9fcf58e5014cbfd4a1e48
8,グルメ,グルメ,50,3500,1750,2011-08-11 12:00:00,2011-08-13 12:00:00,2,2011-08-14,2011-11-14,...,1.0,1.0,1.0,1.0,1.0,1.0,関東,東京都,恵比寿・目黒・品川,74e3a12659d38ef20378f156cfeaf204
9,グルメ,グルメ,57,10500,4500,2011-09-21 12:00:00,2011-09-24 12:00:00,3,2011-09-26,2012-02-29,...,0.0,0.0,1.0,0.0,0.0,1.0,関東,東京都,銀座・新橋・東京・上野,8a0ab8c5dd5014df8b4860233238ed44


In [35]:
coupon_list_train.describe()

,PRICE_RATE,CATALOG_PRICE,DISCOUNT_PRICE,DISPPERIOD,VALIDPERIOD,USABLE_DATE_MON,USABLE_DATE_TUE,USABLE_DATE_WED,USABLE_DATE_THU,USABLE_DATE_FRI,USABLE_DATE_SAT,USABLE_DATE_SUN,USABLE_DATE_HOLIDAY,USABLE_DATE_BEFORE_HOLIDAY
count,19413.000000,19413.000000,19413.000000,19413.000000,13266.000000,12408.000000,12408.000000,12408.000000,12408.000000,12408.000000,12408.000000,12408.000000,12408.000000,12408.000000
mean,58.478391,11818.368258,4332.877659,3.166950,125.955902,0.936009,0.953901,0.982189,0.981867,1.000242,1.067618,0.921422,0.938024,1.044246
std,11.266571,16881.898880,5459.667448,1.346859,46.599249,0.309854,0.314045,0.217525,0.198105,0.319562,0.563260,0.424181,0.399619,0.534511
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,50.000000,3675.000000,1550.000000,2.000000,89.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
50%,53.000000,6500.000000,2750.000000,3.000000,128.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
75%,65.000000,13650.000000,4800.000000,4.000000,177.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,100.000000,680000.000000,100000.000000,36.000000,179.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000


In [36]:
coupon_purchases_train

,ITEM_COUNT,I_DATE,SMALL_AREA_NAME,PURCHASEID_hash,USER_ID_hash,COUPON_ID_hash
0,1,2012-03-28 15:06:06,兵庫,c820a8882374a4e472f0984a8825893f,d9dca3cb44bab12ba313eaa681f663eb,34c48f84026e08355dc3bd19b427f09a
1,1,2011-07-04 23:52:54,銀座・新橋・東京・上野,1b4eb2435421ede98c8931c42e8220ec,560574a339f1b25e57b0221e486907ed,767673b7a777854a92b73b0934ddfae7
2,1,2011-07-16 00:52:49,恵比寿・目黒・品川,36b5f9ba46c44b65587d0b16f2e4c77f,560574a339f1b25e57b0221e486907ed,4f3b5b91d9831192557c056022fdc1f2
3,1,2011-07-16 00:54:53,恵比寿・目黒・品川,2f30f46937cc9004774e576914b2aa1a,560574a339f1b25e57b0221e486907ed,4f3b5b91d9831192557c056022fdc1f2
4,1,2011-07-16 00:55:52,恵比寿・目黒・品川,4d000c64a55ac573d0ae1a8f03677f50,560574a339f1b25e57b0221e486907ed,4f3b5b91d9831192557c056022fdc1f2
5,1,2011-07-16 00:57:09,恵比寿・目黒・品川,d8b030c8a4a2c1051997092a2985b373,560574a339f1b25e57b0221e486907ed,4f3b5b91d9831192557c056022fdc1f2
6,2,2011-07-16 00:58:29,恵比寿・目黒・品川,2c98138766edf5d5be97ed96aa188c7d,560574a339f1b25e57b0221e486907ed,4f3b5b91d9831192557c056022fdc1f2
7,2,2011-09-07 00:04:47,渋谷・青山・自由が丘,5a45d62c72a4cc4155786b03bec74c96,560574a339f1b25e57b0221e486907ed,259a22533cab381464c89f616a56d12d
8,1,2011-10-19 22:21:50,渋谷・青山・自由が丘,592bb67c0553648d6ebd99a4442606f8,560574a339f1b25e57b0221e486907ed,f4a6f861a266799b07cd99c26e80962b
9,1,2011-12-01 16:53:45,恵比寿・目黒・品川,509883c481716b1f3abeac8372b36a5d,560574a339f1b25e57b0221e486907ed,4257a8a169b40bbbcb002ec47ab3de13


In [37]:
coupon_purchases_train.describe()

,ITEM_COUNT
count,168996.000000
mean,1.430939
std,0.982732
min,1.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,55.000000


In [38]:
purchased_coupons_train = coupon_purchases_train.merge(coupon_list_train, on='COUPON_ID_hash',
                                                 how='inner')

In [39]:
purchased_coupons_train

,ITEM_COUNT,I_DATE,SMALL_AREA_NAME,PURCHASEID_hash,USER_ID_hash,COUPON_ID_hash,CAPSULE_TEXT,GENRE_NAME,PRICE_RATE,CATALOG_PRICE,...,USABLE_DATE_WED,USABLE_DATE_THU,USABLE_DATE_FRI,USABLE_DATE_SAT,USABLE_DATE_SUN,USABLE_DATE_HOLIDAY,USABLE_DATE_BEFORE_HOLIDAY,large_area_name,ken_name,small_area_name
0,1,2012-03-28 15:06:06,兵庫,c820a8882374a4e472f0984a8825893f,d9dca3cb44bab12ba313eaa681f663eb,34c48f84026e08355dc3bd19b427f09a,宅配,宅配,78,7200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
1,1,2012-03-26 23:26:24,千葉,028ac37f96a17e6be2db51ed0884ac5a,dd8a79645e8170db6a91bad1d89b4055,34c48f84026e08355dc3bd19b427f09a,宅配,宅配,78,7200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
2,1,2012-03-26 13:50:20,新宿・高田馬場・中野・吉祥寺,eb7f867c9f1b14c1a2b1cc47004828ae,658249379aaaf2b2320061e5218c0c2c,34c48f84026e08355dc3bd19b427f09a,宅配,宅配,78,7200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
3,1,2012-03-28 22:46:58,和歌山,fddda2bdaba87fc9b86d97e908152014,1c4aad19f033d97dc163b481116de322,34c48f84026e08355dc3bd19b427f09a,宅配,宅配,78,7200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
4,1,2012-03-29 15:09:51,宮崎,6659c8fdb5f66b4013613ff770f91db7,e78ae0887c4a9dce91ff0d2a45a9f72c,34c48f84026e08355dc3bd19b427f09a,宅配,宅配,78,7200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
5,1,2012-03-29 12:42:37,千葉,be5af03efe7e4bc2f1e7e8317531cd74,39cabe60f73ea8cad96d7f895ea944ee,34c48f84026e08355dc3bd19b427f09a,宅配,宅配,78,7200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
6,1,2012-03-28 22:45:16,和歌山,aae8c29627af3489c530f141eb02a3a1,fca1e37548c9ca5415a9850a1b9b2c28,34c48f84026e08355dc3bd19b427f09a,宅配,宅配,78,7200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
7,1,2012-03-29 18:10:28,栃木,ea4cf60407a03246be6c588c12542780,3511c63ae9ce45fc3b5cf5054956d5a4,34c48f84026e08355dc3bd19b427f09a,宅配,宅配,78,7200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
8,1,2012-03-28 00:05:32,栃木,448cec2c62179725c9ac0d93006074d0,77b92939627690028e22834d7920bf4e,34c48f84026e08355dc3bd19b427f09a,宅配,宅配,78,7200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
9,2,2012-03-27 11:47:59,栃木,f8f426b0f2beb05978e856a727febac8,1ce49ceb6f55d3b3ed3180f811ec3ffb,34c48f84026e08355dc3bd19b427f09a,宅配,宅配,78,7200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺


In [40]:
features = ['COUPON_ID_hash', 'USER_ID_hash',
            'GENRE_NAME', 'DISCOUNT_PRICE', 'PRICE_RATE',
            'USABLE_DATE_MON', 'USABLE_DATE_TUE', 'USABLE_DATE_WED', 'USABLE_DATE_THU',
            'USABLE_DATE_FRI', 'USABLE_DATE_SAT', 'USABLE_DATE_SUN', 'USABLE_DATE_HOLIDAY',
            'USABLE_DATE_BEFORE_HOLIDAY', 'large_area_name', 'ken_name', 'small_area_name']

In [41]:
purchased_coupons_train = purchased_coupons_train[features]

In [42]:
coupon_list_test['USER_ID_hash'] = 'dummyuser'

In [43]:
coupon_list_test = coupon_list_test[features]

In [44]:
combined = pd.concat([purchased_coupons_train, coupon_list_test], axis=0)

In [45]:
combined['DISCOUNT_PRICE'] = 1 / np.log10(combined['DISCOUNT_PRICE'])
combined['PRICE_RATE'] = (combined['PRICE_RATE'] / 100) ** 2
features.extend(['DISCOUNT_PRICE', 'PRICE_RATE'])

/home/nunoferreira/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log10
  """Entry point for launching an IPython kernel.


In [46]:
categoricals = ['GENRE_NAME', 'USABLE_DATE_MON', 'USABLE_DATE_TUE', 'USABLE_DATE_WED',
                'USABLE_DATE_THU', 'USABLE_DATE_FRI', 'USABLE_DATE_SAT', 'USABLE_DATE_SUN',
                'USABLE_DATE_HOLIDAY', 'USABLE_DATE_BEFORE_HOLIDAY', 'large_area_name', 'ken_name', 'small_area_name']
combined_categoricals = combined[categoricals]
combined_categoricals = pd.get_dummies(combined_categoricals,
                                    dummy_na=False)

In [47]:
continuous = list(set(features) - set(categoricals))
combined = pd.concat([combined[continuous], combined_categoricals], axis=1)

In [48]:
NAN_SUBSTITUTION_VALUE = 1
combined = combined.fillna(NAN_SUBSTITUTION_VALUE)

In [49]:
train = combined[combined['USER_ID_hash'] != 'dummyuser']
test = combined[combined['USER_ID_hash'] == 'dummyuser']
test.drop('USER_ID_hash', inplace=True, axis=1)

/home/nunoferreira/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [50]:
train_dropped_coupons = train.drop('COUPON_ID_hash', axis=1)
user_profiles = train_dropped_coupons.groupby(by='USER_ID_hash').apply(np.mean)

In [51]:
FEATURE_WEIGHTS = {
    'GENRE_NAME': 2,
    'DISCOUNT_PRICE': 2,
    'PRICE_RATE': 0,
    'USABLE_DATE_': 0,
    'large_area_name': 0.5,
    'ken_name': 1,
    'small_area_name': 5
}

In [52]:
def find_appropriate_weight(weights_dict, colname):
    for col, weight in weights_dict.items():
        if col in colname:
            return weight
    raise ValueError

W_values = [find_appropriate_weight(FEATURE_WEIGHTS, colname)
        for colname in user_profiles.columns]
W = np.diag(W_values)

In [53]:
test_only_features = test.drop('COUPON_ID_hash', axis=1)
similarity_scores = np.dot(np.dot(user_profiles, W), test_only_features.T)

In [54]:
coupons_ids = test['COUPON_ID_hash']
index = user_profiles.index
columns = [coupons_ids[i] for i in range(0, similarity_scores.shape[1])]
result_df = pd.DataFrame(index=index, columns=columns, data=similarity_scores)

In [55]:
result_df

,c76ea297ebd3a5a4d3bf9f75269f66fa,dd74dc95ca294afa02db40a543ae1763,c65b550cbef918796ad53b1d5b7165c1,f93dc6e223935d817e1237f8f73b56a2,86c64391318f1d751647bf8e1882cf1d,78aec0e250021cf5247986f6ca55a47e,44000991cad71e5105d3ad3b78d92431,3810431a7769cfcc3201383b5e83248c,520cb8509c6662059d3f71e7c3cad0ae,db25ee0094a086ed6663f94e41428835,...,9e5f11a475b172a6384dec160865fcc2,6adfd771501bf0595c0538f6eb60a84a,0c015306597566b632bebfb63b7e59f3,9cfcf41925ff2b183ff55789462333a3,c988d799bc7db9254fe865ee6cf2d4ff,65500c6708fa82d36b3a467a94dd2c50,e3e9027e1b87d957b1ccdb24d128e455,09ec991661af3ac7bc3b5cddc31e2fa0,73675e451067962afc48257474308cd0,f9c657ce7ca80b3766ced3a9a3c709bb
USER_ID_hash,,,,,,,,,,,,,,,,,,,,,
0000b53e182165208887ba65c079fc21,8.667846,2.146502,2.162546,2.678919,2.179665,2.151518,2.153973,2.163011,2.202337,2.165698,...,0.149715,2.170489,0.287835,0.138845,0.287835,2.694862,3.694862,2.694862,0.158882,0.638845
00035b86e6884589ec8d28fbf2fe7757,2.183769,2.160401,2.177966,2.195893,2.196710,2.165892,2.168579,2.178475,2.221532,2.181417,...,0.163918,2.186663,0.315141,0.152017,0.315141,2.213348,2.213348,2.213348,0.173955,0.152017
0005b1068d5f2b8f2a7c978fcfe1ca06,0.213642,1.686475,0.706896,0.227737,0.228686,1.692859,1.695983,6.707488,1.757544,1.710908,...,1.690564,1.717006,7.866369,1.676728,6.866369,0.248029,0.248029,0.248029,1.702232,0.176728
000cc06982785a19e2a2fdb40b1c9d59,1.527326,1.907419,1.021200,0.682981,0.493367,1.913216,1.916053,2.164595,1.971952,1.929605,...,0.434942,1.935142,2.594577,1.612855,2.308863,0.701407,2.510931,0.701407,1.636013,0.350950
0013518e41c416cd6a181d277dd8ca0b,0.227804,1.323836,0.595611,0.242833,1.868846,1.330644,1.333975,5.096242,1.399617,1.349889,...,0.578197,1.356392,5.265656,1.813444,6.265656,0.264471,0.264471,0.264471,1.340638,0.688444
001acdee812a18acfd7509172bed5700,1.473089,0.716397,0.428191,0.871154,0.256676,0.722489,0.725470,1.505678,0.784212,1.124326,...,0.874145,0.745530,1.811130,1.091712,1.811130,0.890517,1.505902,0.428979,0.731433,0.360943
001fd7876e3aa29393537c6baf308e43,0.179268,6.656472,0.673608,0.191095,0.191892,6.661829,6.664451,1.674104,6.716107,1.676974,...,0.659904,6.682091,1.807422,1.648294,1.807422,0.208123,0.208123,0.208123,6.669694,0.148294
002383753c1e5d6305c8aff6f89e26d6,2.194377,2.169660,2.188240,2.207201,2.208065,2.175469,2.178311,2.188778,2.234321,2.191890,...,0.173381,2.197439,0.333333,0.160793,0.333333,2.225664,2.225664,2.225664,0.183997,0.160793
0025cae7997d25ea5cf8851bb099c798,1.525198,3.445245,2.685806,1.537856,1.538709,3.450978,3.453784,5.686338,3.509070,2.911631,...,0.671139,3.472664,4.495691,1.547603,4.717913,1.556080,1.556080,1.556080,2.126063,0.158714


In [56]:
train_dropped_coupons = train.drop(['COUPON_ID_hash', 'USER_ID_hash'], axis=1)

In [57]:
cosine_similarity(train_dropped_coupons[:100], train_dropped_coupons[:100])

array([[1.        , 1.        , 1.        , ..., 0.92012483, 0.92012483,
        0.92012483],
       [1.        , 1.        , 1.        , ..., 0.92012483, 0.92012483,
        0.92012483],
       [1.        , 1.        , 1.        , ..., 0.92012483, 0.92012483,
        0.92012483],
       ...,
       [0.92012483, 0.92012483, 0.92012483, ..., 1.        , 1.        ,
        1.        ],
       [0.92012483, 0.92012483, 0.92012483, ..., 1.        , 1.        ,
        1.        ],
       [0.92012483, 0.92012483, 0.92012483, ..., 1.        , 1.        ,
        1.        ]])